In [3]:
import requests
import pandas as pd

In [28]:
#Fetch Data from FPL API
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
data = response.json()

In [31]:
# Extract Team Data
teams_data = data['teams']
teams = pd.DataFrame(teams_data)

# Clean and explore team data
teams_cleaned = teams[['id', 'name', 'strength_attack_home', 'strength_attack_away', 
                       'strength_defence_home', 'strength_defence_away', 'strength_overall_home', 
                       'strength_overall_away', 'form', 'team_division']]

In [33]:
# Extract player data
players_data = data['elements']
players = pd.DataFrame(players_data)

# Extract relevant columns for Player-Level Stats
players_cleaned = players[['id', 'web_name', 'team', 'expected_goals', 'expected_assists', 
                           'minutes', 'form', 'news']]

In [35]:
# Filter Active Players
# Remove players who have 0 minutes or have injury flags (e.g. 'news' is not empty)
players_active = players_cleaned[players_cleaned['minutes'] > 0]
players_active = players_active[players_active['news'].isna()]  # Filter out injured players

In [39]:
# Calculate Averages for Team Stats
teams_cleaned.loc[:, 'average_attack_strength'] = (teams_cleaned['strength_attack_home'] + teams_cleaned['strength_attack_away']) / 2
teams_cleaned.loc[:, 'average_defence_strength'] = (teams_cleaned['strength_defence_home'] + teams_cleaned['strength_defence_away']) / 2

# Apply form conversion to numeric (float) and handle missing values
teams_cleaned.loc[:, 'average_form'] = teams_cleaned['form'].apply(lambda x: float(x) if x else 0)


In [45]:
# Aggregate player stats for each team
team_stats = []
for team_id in teams_cleaned['id']:
    # Filter players for the current team
    team_players = players_active[players_active['team'] == team_id]
    
    # If the team has no active players, set stats to 0
    if team_players.empty:
        avg_xg = avg_xa = avg_minutes = 0
    else:
        # Calculate average xG, xA, and minutes for the team
        avg_xg = team_players['expected_goals'].mean()
        avg_xa = team_players['expected_assists'].mean()
        avg_minutes = team_players['minutes'].mean()

    # Append stats to team_stats
    team_stats.append({
        'team_id': team_id,
        'average_xg': avg_xg,
        'average_xa': avg_xa,
        'average_minutes': avg_minutes,
    })

team_stats_df = pd.DataFrame(team_stats)

# Merge Team-Level Stats with Aggregated Player Stats
teams_stats_final = pd.merge(teams_cleaned, team_stats_df, left_on='id', right_on='team_id', how='left')

# Display the final DataFrame
print(teams_stats_final[['name', 'average_attack_strength', 'average_defence_strength', 'average_form', 
                         'average_xg', 'average_xa', 'average_minutes']].head())


          name  average_attack_strength  average_defence_strength  \
0      Arsenal                   1315.0                    1295.0   
1  Aston Villa                   1120.0                    1230.0   
2  Bournemouth                   1110.0                    1200.0   
3    Brentford                   1085.0                    1220.0   
4     Brighton                   1090.0                    1170.0   

   average_form  average_xg  average_xa  average_minutes  
0             0           0           0                0  
1             0           0           0                0  
2             0           0           0                0  
3             0           0           0                0  
4             0           0           0                0  


In [ ]:
print(teams_cleaned[['id', 'name']].head())
print(team_stats_df.head())

In [47]:
# Step 7: Display the Cleaned and Processed Data
print(teams_stats_final[['name', 'average_attack_strength', 'average_defence_strength', 'average_form', 
                         'average_xg', 'average_xa', 'average_minutes']].head())

          name  average_attack_strength  average_defence_strength  \
0      Arsenal                   1315.0                    1295.0   
1  Aston Villa                   1120.0                    1230.0   
2  Bournemouth                   1110.0                    1200.0   
3    Brentford                   1085.0                    1220.0   
4     Brighton                   1090.0                    1170.0   

   average_form  average_xg  average_xa  average_minutes  
0             0           0           0                0  
1             0           0           0                0  
2             0           0           0                0  
3             0           0           0                0  
4             0           0           0                0  
